In [1]:
import pandas as pd
import numpy as np

In [9]:
crime_data = pd.read_csv('../new_data/zip_crime_data.csv')


In [10]:
crime_data = one_hot_decoding(crime_data)

0 out of 345438
1 out of 345438
2 out of 345438
3 out of 345438
4 out of 345438
5 out of 345438
6 out of 345438
7 out of 345438
8 out of 345438
9 out of 345438
10 out of 345438
11 out of 345438
12 out of 345438
13 out of 345438
14 out of 345438
15 out of 345438
16 out of 345438
17 out of 345438
18 out of 345438
19 out of 345438
20 out of 345438
21 out of 345438
22 out of 345438
23 out of 345438
24 out of 345438
25 out of 345438
26 out of 345438
27 out of 345438
28 out of 345438
29 out of 345438
30 out of 345438
31 out of 345438
32 out of 345438
33 out of 345438
34 out of 345438
35 out of 345438
36 out of 345438
37 out of 345438
38 out of 345438
39 out of 345438
40 out of 345438
41 out of 345438
42 out of 345438
43 out of 345438
44 out of 345438
45 out of 345438
46 out of 345438
47 out of 345438
48 out of 345438
49 out of 345438
50 out of 345438
51 out of 345438
52 out of 345438
53 out of 345438
54 out of 345438
55 out of 345438
56 out of 345438
57 out of 345438
58 out of 345438
59 out 

463 out of 345438
464 out of 345438
465 out of 345438
466 out of 345438
467 out of 345438
468 out of 345438
469 out of 345438
470 out of 345438
471 out of 345438
472 out of 345438
473 out of 345438
474 out of 345438
475 out of 345438
476 out of 345438
477 out of 345438
478 out of 345438
479 out of 345438
480 out of 345438
481 out of 345438
482 out of 345438
483 out of 345438
484 out of 345438
485 out of 345438
486 out of 345438
487 out of 345438
488 out of 345438
489 out of 345438
490 out of 345438
491 out of 345438
492 out of 345438
493 out of 345438
494 out of 345438
495 out of 345438
496 out of 345438
497 out of 345438
498 out of 345438
499 out of 345438
500 out of 345438
501 out of 345438
502 out of 345438
503 out of 345438
504 out of 345438
505 out of 345438
506 out of 345438
507 out of 345438
508 out of 345438
509 out of 345438
510 out of 345438
511 out of 345438
512 out of 345438
513 out of 345438
514 out of 345438
515 out of 345438
516 out of 345438
517 out of 345438
518 out of

920 out of 345438
921 out of 345438
922 out of 345438
923 out of 345438
924 out of 345438
925 out of 345438
926 out of 345438
927 out of 345438
928 out of 345438
929 out of 345438
930 out of 345438
931 out of 345438
932 out of 345438
933 out of 345438
934 out of 345438
935 out of 345438
936 out of 345438
937 out of 345438
938 out of 345438
939 out of 345438
940 out of 345438
941 out of 345438
942 out of 345438
943 out of 345438
944 out of 345438
945 out of 345438
946 out of 345438
947 out of 345438
948 out of 345438
949 out of 345438
950 out of 345438
951 out of 345438
952 out of 345438
953 out of 345438
954 out of 345438
955 out of 345438
956 out of 345438
957 out of 345438
958 out of 345438
959 out of 345438
960 out of 345438
961 out of 345438
962 out of 345438
963 out of 345438
964 out of 345438
965 out of 345438
966 out of 345438
967 out of 345438
968 out of 345438
969 out of 345438
970 out of 345438
971 out of 345438
972 out of 345438
973 out of 345438
974 out of 345438
975 out of

1357 out of 345438
1358 out of 345438
1359 out of 345438
1360 out of 345438
1361 out of 345438
1362 out of 345438
1363 out of 345438
1364 out of 345438
1365 out of 345438
1366 out of 345438
1367 out of 345438
1368 out of 345438
1369 out of 345438
1370 out of 345438
1371 out of 345438
1372 out of 345438
1373 out of 345438
1374 out of 345438
1375 out of 345438
1376 out of 345438
1377 out of 345438
1378 out of 345438
1379 out of 345438
1380 out of 345438
1381 out of 345438
1382 out of 345438
1383 out of 345438
1384 out of 345438
1385 out of 345438
1386 out of 345438
1387 out of 345438
1388 out of 345438
1389 out of 345438
1390 out of 345438
1391 out of 345438
1392 out of 345438
1393 out of 345438
1394 out of 345438
1395 out of 345438
1396 out of 345438
1397 out of 345438
1398 out of 345438
1399 out of 345438
1400 out of 345438
1401 out of 345438
1402 out of 345438
1403 out of 345438
1404 out of 345438
1405 out of 345438
1406 out of 345438
1407 out of 345438
1408 out of 345438
1409 out of 

1791 out of 345438
1792 out of 345438
1793 out of 345438
1794 out of 345438
1795 out of 345438
1796 out of 345438
1797 out of 345438
1798 out of 345438
1799 out of 345438
1800 out of 345438
1801 out of 345438
1802 out of 345438
1803 out of 345438
1804 out of 345438
1805 out of 345438
1806 out of 345438
1807 out of 345438
1808 out of 345438
1809 out of 345438
1810 out of 345438
1811 out of 345438
1812 out of 345438
1813 out of 345438
1814 out of 345438
1815 out of 345438
1816 out of 345438
1817 out of 345438
1818 out of 345438
1819 out of 345438
1820 out of 345438
1821 out of 345438
1822 out of 345438
1823 out of 345438
1824 out of 345438
1825 out of 345438
1826 out of 345438
1827 out of 345438
1828 out of 345438
1829 out of 345438
1830 out of 345438
1831 out of 345438
1832 out of 345438
1833 out of 345438
1834 out of 345438
1835 out of 345438
1836 out of 345438
1837 out of 345438
1838 out of 345438
1839 out of 345438
1840 out of 345438
1841 out of 345438
1842 out of 345438
1843 out of 

...

3959 out of 345438
3960 out of 345438
3961 out of 345438
3962 out of 345438
3963 out of 345438
3964 out of 345438
3965 out of 345438
3966 out of 345438
3967 out of 345438
3968 out of 345438
3969 out of 345438
3970 out of 345438
3971 out of 345438
3972 out of 345438
3973 out of 345438
3974 out of 345438
3975 out of 345438
3976 out of 345438
3977 out of 345438
3978 out of 345438
3979 out of 345438
3980 out of 345438
3981 out of 345438
3982 out of 345438
3983 out of 345438
3984 out of 345438
3985 out of 345438
3986 out of 345438
3987 out of 345438
3988 out of 345438
3989 out of 345438
3990 out of 345438
3991 out of 345438
3992 out of 345438
3993 out of 345438
3994 out of 345438
3995 out of 345438
3996 out of 345438
3997 out of 345438
3998 out of 345438
3999 out of 345438
4000 out of 345438
4001 out of 345438
4002 out of 345438
4003 out of 345438
4004 out of 345438
4005 out of 345438
4006 out of 345438
4007 out of 345438
4008 out of 345438
4009 out of 345438
4010 out of 345438
4011 out of 

4395 out of 345438
4396 out of 345438
4397 out of 345438
4398 out of 345438
4399 out of 345438
4400 out of 345438
4401 out of 345438
4402 out of 345438
4403 out of 345438
4404 out of 345438
4405 out of 345438
4406 out of 345438
4407 out of 345438
4408 out of 345438
4409 out of 345438
4410 out of 345438
4411 out of 345438
4412 out of 345438
4413 out of 345438
4414 out of 345438
4415 out of 345438
4416 out of 345438
4417 out of 345438
4418 out of 345438
4419 out of 345438
4420 out of 345438
4421 out of 345438
4422 out of 345438
4423 out of 345438
4424 out of 345438
4425 out of 345438
4426 out of 345438
4427 out of 345438
4428 out of 345438
4429 out of 345438
4430 out of 345438
4431 out of 345438
4432 out of 345438
4433 out of 345438
4434 out of 345438
4435 out of 345438
4436 out of 345438
4437 out of 345438
4438 out of 345438
4439 out of 345438
4440 out of 345438
4441 out of 345438
4442 out of 345438
4443 out of 345438
4444 out of 345438
4445 out of 345438
4446 out of 345438
4447 out of 

4829 out of 345438
4830 out of 345438
4831 out of 345438
4832 out of 345438
4833 out of 345438
4834 out of 345438
4835 out of 345438
4836 out of 345438
4837 out of 345438
4838 out of 345438
4839 out of 345438
4840 out of 345438
4841 out of 345438
4842 out of 345438
4843 out of 345438
4844 out of 345438
4845 out of 345438
4846 out of 345438
4847 out of 345438
4848 out of 345438
4849 out of 345438
4850 out of 345438
4851 out of 345438
4852 out of 345438
4853 out of 345438
4854 out of 345438
4855 out of 345438
4856 out of 345438
4857 out of 345438
4858 out of 345438
4859 out of 345438
4860 out of 345438
4861 out of 345438
4862 out of 345438
4863 out of 345438
4864 out of 345438
4865 out of 345438
4866 out of 345438
4867 out of 345438
4868 out of 345438
4869 out of 345438
4870 out of 345438
4871 out of 345438
4872 out of 345438
4873 out of 345438
4874 out of 345438
4875 out of 345438
4876 out of 345438
4877 out of 345438
4878 out of 345438
4879 out of 345438
4880 out of 345438
4881 out of 

5265 out of 345438
5266 out of 345438
5267 out of 345438
5268 out of 345438
5269 out of 345438
5270 out of 345438
5271 out of 345438
5272 out of 345438
5273 out of 345438
5274 out of 345438
5275 out of 345438
5276 out of 345438
5277 out of 345438
5278 out of 345438
5279 out of 345438
5280 out of 345438
5281 out of 345438
5282 out of 345438
5283 out of 345438
5284 out of 345438
5285 out of 345438
5286 out of 345438
5287 out of 345438
5288 out of 345438
5289 out of 345438
5290 out of 345438
5291 out of 345438
5292 out of 345438
5293 out of 345438
5294 out of 345438
5295 out of 345438
5296 out of 345438
5297 out of 345438
5298 out of 345438
5299 out of 345438
5300 out of 345438
5301 out of 345438
5302 out of 345438
5303 out of 345438
5304 out of 345438
5305 out of 345438
5306 out of 345438
5307 out of 345438
5308 out of 345438
5309 out of 345438
5310 out of 345438
5311 out of 345438
5312 out of 345438
5313 out of 345438
5314 out of 345438
5315 out of 345438
5316 out of 345438
5317 out of 

KeyboardInterrupt: 

In [ ]:
crime_data

In [3]:
raw_restaurant_data = pd.read_csv('../new_data/restaurant_data.csv')
restaurant_zip_sum_data = raw_restaurant_data.groupby(['zipcode']).sum().rename(index=int, columns={'count':'res_total_count'}).reset_index()
restaurant_zip_max_data = raw_restaurant_data.groupby(['zipcode']).max().rename(index=int, columns={'count':'res_max_type_count', 'type':'res_max_type'}).reset_index()
restaurant_data = restaurant_zip_max_data.join(restaurant_zip_sum_data.set_index('zipcode'), on='zipcode')

In [4]:
ny_house_sell_data = pd.read_csv('../new_data/ny_house_sell.csv')
ny_house_sell_data['avg_house_sell_price'] = ny_house_sell_data.drop('zipcode', axis='columns').mean(axis=1)
ny_house_sell_data = ny_house_sell_data[['zipcode', 'avg_house_sell_price']]
ny_house_rent_data = pd.read_csv('../new_data/ny_house_rent.csv')
ny_house_rent_data['avg_house_rent_price'] = ny_house_rent_data.drop('zipcode', axis='columns').mean(axis=1)
ny_house_rent_data = ny_house_rent_data[['zipcode', 'avg_house_rent_price']]

In [5]:
lib_data = pd.read_csv('../data/NYC_Lib_zip_count.csv')\
                .rename(index=int, columns={"count1": "lib_count1", "count5": "lib_count5", "count15":"lib_count15"})

In [6]:
mus_data = pd.read_csv('../data/NYC_Mus_zip_count.csv')\
                .rename(index=int, columns={"count1": "mus_count1", "count5": "mus_count5", "count15":"mus_count15"})

In [7]:
def my_stupid_approach(store_data):
    new_data = store_data.groupby('Address ZIP').sum().reset_index()
    new_data['store_business_type_count'] = new_data['store_individual_type_count'] = 0
    for index, row in store_data.iterrows():
        if row['store_type'] == 'Business':
            new_data.loc[new_data['Address ZIP'] == row['Address ZIP'], 'store_business_type_count'] = row['store_count']
        else:
            new_data.loc[new_data['Address ZIP'] == row['Address ZIP'], 'store_individual_type_count'] = row['store_count']
    return new_data

store_data = pd.read_csv('../data/NYC_Store_Count.csv')\
        .rename(index=int, columns={"License Type": "store_type", "DCA License Number":"store_count"})
store_data = my_stupid_approach(store_data)[['Address ZIP', 'store_business_type_count', 'store_individual_type_count']]

In [8]:
univ_data = pd.read_csv('../data/NYC_Univ_zip_count.csv')\
        .rename(index=int, columns={"count1": "univ_count1", "count5": "univ_count5", "count15":"univ_count15"})

In [9]:
popu_data = pd.read_csv('../data/popu_zip.csv')[['zipcode', 'population']]

In [10]:
hospital_data = pd.read_csv('../data/NYC_hospital_zip_count.csv')\
                .rename(index=int, columns={"count10": "hospital_count10", "count5": "hospital_count5", "count15":"hospital_count15"})

In [11]:
school_data = pd.read_csv('../data/NYC_school_zip_count.csv')\
                    .rename(index=int, columns={"count":"school_count"}).groupby('Address ZIP')['school_count'].sum().reset_index()

In [12]:
park_data = pd.read_csv('../data/NYU_parkArea_zip.csv')\
                    .rename(index=str, columns={"count":"park_area"})

In [13]:
dataset = crime_data
dataset = dataset.join(restaurant_data.set_index('zipcode'), on='zipcode')
dataset = dataset.join(ny_house_sell_data.set_index('zipcode'), on='zipcode')
dataset = dataset.join(ny_house_rent_data.set_index('zipcode'), on='zipcode')
dataset = dataset.join(lib_data.set_index('Address ZIP'), on='zipcode')
dataset = dataset.join(mus_data.set_index('Address ZIP'), on='zipcode')
dataset = dataset.join(store_data.set_index('Address ZIP'), on='zipcode')
dataset = dataset.join(univ_data.set_index('Address ZIP'), on='zipcode')
dataset = dataset.join(popu_data.set_index('zipcode'), on='zipcode')
dataset = dataset.join(hospital_data.set_index('Address ZIP'), on='zipcode')
# dataset = dataset.join(school_data.set_index('Address ZIP'), on='zipcode')
dataset = dataset.join(park_data.set_index('Address ZIP'), on='zipcode')

In [14]:
dataset.head(1).transpose()

,0
date,09/30/2017
time,23:46:00
lat,40.8011
lon,-73.9505
LEVEL_FELONY,1
LEVEL_MISDEMEANOR,0
LEVEL_VIOLATION,0
zipcode,10026
res_max_type,Vegetarian
res_max_type_count,194


In [15]:
dataset = dataset.fillna({'res_max_type':'Unknown',
                          'res_max_type_count': 0, 
                          'res_total_count':0, 
                          'avg_house_sell_price':dataset['avg_house_sell_price'].mean(), 
                          'avg_house_rent_price':dataset['avg_house_rent_price'].mean(), 
                          'lib_count1':0, 
                          'lib_count5':0, 
                          'lib_count15':0, 
                          'mus_count1':0, 
                          'mus_count5':0, 
                          'mus_count15':0, 
                          'store_business_type_count':0, 
                          'store_individual_type_count':0, 
                          'univ_count1':0, 
                          'univ_count5':0, 
                          'univ_count15':0, 
                          'population':dataset['population'].mean(), 
                          'hospital_count5':0, 
                          'hospital_count10':0, 
                          'hospital_count15':0, 
                          'school_count':0,
                          'park_area':0})

In [16]:
dataset

,date,time,lat,lon,LEVEL_FELONY,LEVEL_MISDEMEANOR,LEVEL_VIOLATION,zipcode,res_max_type,res_max_type_count,...,store_business_type_count,store_individual_type_count,univ_count1,univ_count5,univ_count15,population,hospital_count5,hospital_count10,hospital_count15,park_area
0,09/30/2017,23:46:00,40.801064,-73.950482,1,0,0,10026,Vegetarian,194.0,...,118.0,134.0,0.0,17.0,124.0,34003.00000,14.0,64.0,130.0,569.897094
1,09/30/2017,23:20:00,40.677407,-74.006397,0,1,0,11231,Vietnamese/Cambodian/Malaysia,456.0,...,202.0,89.0,0.0,1.0,11.0,33336.00000,0.0,0.0,0.0,1410.767771
2,09/30/2017,23:20:00,40.623227,-74.149227,0,1,0,10302,Spanish,98.0,...,166.0,88.0,0.0,3.0,23.0,19088.00000,7.0,20.0,35.0,527.570103
3,09/30/2017,23:20:00,40.654698,-73.907624,0,0,1,11212,Spanish,207.0,...,321.0,96.0,0.0,0.0,0.0,84500.00000,14.0,40.0,82.0,1496.452799
4,09/30/2017,23:15:00,40.747495,-73.885321,1,0,0,11372,Thai,722.0,...,523.0,95.0,0.0,2.0,13.0,66636.00000,15.0,39.0,64.0,263.435729
5,09/30/2017,23:10:00,40.820204,-73.863637,0,1,0,10473,Spanish,60.0,...,113.0,118.0,0.0,0.0,0.0,58519.00000,0.0,0.0,0.0,575.000734
6,09/30/2017,23:10:00,40.770794,-73.933146,1,0,0,11102,Spanish,144.0,...,203.0,105.0,0.0,0.0,8.0,34133.00000,0.0,6.0,23.0,1504.581831
7,09/30/2017,23:09:00,40.694554,-73.802113,0,1,0,11451,American,18.0,...,0.0,0.0,0.0,6.0,40.0,0.00000,6.0,24.0,44.0,347.542281
8,09/30/2017,23:05:00,40.677098,-73.825040,1,0,0,11419,Vegetarian,285.0,...,413.0,252.0,0.0,0.0,10.0,47211.00000,0.0,0.0,6.0,134.107500
9,09/30/2017,23:02:00,40.756645,-73.986285,0,1,0,10018,Vietnamese/Cambodian/Malaysia,932.0,...,321.0,31.0,0.0,7.0,55.0,5229.00000,0.0,0.0,0.0,109.251121


In [17]:
dataset.to_csv('../data/all_data.csv', sep=',', index=False)